In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, date

In [36]:
df = pd.read_csv('Data AnalystGlassdoor_posts(2023-12-23).csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   job_title         820 non-null    object 
 1   company_name      820 non-null    object 
 2   location          820 non-null    object 
 3   days_posted       820 non-null    object 
 4   salary_estimate   719 non-null    object 
 5   job_description   820 non-null    object 
 6   company_rating    820 non-null    float64
 7   company_size      820 non-null    object 
 8   company_founded   820 non-null    object 
 9   company_type      820 non-null    object 
 10  company_industry  820 non-null    object 
 11  company_sector    820 non-null    object 
 12  company_revenue   820 non-null    object 
dtypes: float64(1), object(12)
memory usage: 83.4+ KB


In [37]:
def convert_days_posted_to_date(days_posted: str) -> date:
    today = date.today()

    if days_posted == "24h":
        return today
    elif days_posted.endswith("d"):
        days = int(days_posted[:-1])  # Remove the "d" and convert to int
        return today - timedelta(days=days)
    elif days_posted == "30d+":
        return today - timedelta(days=30)
    else:
        return None  # Return None or raise an exception if the format is unexpected

In [38]:
def salary(df):
    # Extract start and end salaries
    df[['start', 'end']] = df['salary_estimate'].str.extract(r'\$(\d+K) - \$(\d+K)')

    # Remove the 'K' and convert to integer
    df['start'] = df['start'].str.replace('K', '').astype(float) * 1000   
    df['end'] = df['end'].str.replace('K', '').astype(float) * 1000

    # Calculate average
    df['average'] = df[['start', 'end']].mean(axis=1)

    return df

In [39]:
df = df.assign(
    company_size = lambda x: x['company_size'].str.replace('Size', ''),
    company_founded = lambda x: pd.to_numeric(x['company_founded'].str.replace('Founded', ''), errors='coerce').fillna(0).astype(int),
    company_type = lambda x: x['company_type'].str.replace('Type', ''),
    company_industry = lambda x: x['company_industry'].str.replace('Industry', ''),
    company_sector = lambda x: x['company_sector'].str.replace('Sector', ''),
    company_revenue = lambda x: x['company_revenue'].str.replace('Revenue', ''),
    post_date = lambda x: pd.to_datetime(x['days_posted'].apply(convert_days_posted_to_date)),
    age = lambda x:  date.today().year - x['company_founded'],
)
df = salary(df)
df.head()


,job_title,company_name,location,days_posted,salary_estimate,job_description,company_rating,company_size,company_founded,company_type,company_industry,company_sector,company_revenue,post_date,age,start,end,average
0,Junior Data Analyst Apprentice,Evergreen Trading,"New York, NY",22d,$45K (Employer est.),About Evergreen\nEvergreen Trading is a media ...,3.4,Unknown,0,Company - Private,Investment & Asset Management,Financial Services,Unknown / Non-Applicable,2023-12-01,2023,NaN,NaN,NaN
1,Data Analyst,Viant,"Irvine, CA",5d,$57K - $70K (Employer est.),WHAT YOU’LL DO\n\nCome help us build Viant’s i...,4.0,501 to 1000 Employees,1999,Company - Public,Advertising & Public Relations,Media & Communication,Unknown / Non-Applicable,2023-12-18,24,57000.0,70000.0,63500.0
2,Sportsbook Data Analyst,DraftKings,"Boston, MA",24h,$77K - $115K (Employer est.),We’re defining what it means to build and deli...,4.0,1001 to 5000 Employees,2012,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable,2023-12-23,11,77000.0,115000.0,96000.0
3,Junior Data Analyst,391 Financial,"Columbia, MO",30d+,$46K - $75K (Glassdoor est.),Junior Data Analyst Responsibilities:\nInterpr...,3.0,1 to 50 Employees,0,Contract,--,--,Unknown / Non-Applicable,2023-11-23,2023,46000.0,75000.0,60500.0
4,Junior Data Analyst,"Paul, Weiss, Rifkind, Wharton & Garrison LLP",United States,24h,$70K - $85K (Employer est.),Job Description\nJunior Data Analyst will be r...,4.0,1001 to 5000 Employees,1946,Private Practice / Firm,Legal,Legal,$100 to $500 million (USD),2023-12-23,77,70000.0,85000.0,77500.0


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   job_title         820 non-null    object        
 1   company_name      820 non-null    object        
 2   location          820 non-null    object        
 3   days_posted       820 non-null    object        
 4   salary_estimate   719 non-null    object        
 5   job_description   820 non-null    object        
 6   company_rating    820 non-null    float64       
 7   company_size      820 non-null    object        
 8   company_founded   820 non-null    int32         
 9   company_type      820 non-null    object        
 10  company_industry  820 non-null    object        
 11  company_sector    820 non-null    object        
 12  company_revenue   820 non-null    object        
 13  post_date         820 non-null    datetime64[ns]
 14  age               820 non-

In [41]:
new_df = df[(df["salary_estimate"].notnull()) & (df["start"].isnull())][["salary_estimate", "start", "end", "average"]]
new_df.head()

,salary_estimate,start,end,average
0,$45K (Employer est.),NaN,NaN,NaN
7,$24.00 - $28.00 Per Hour (Employer est.),NaN,NaN,NaN
17,$20.50 - $28.97 Per Hour (Employer est.),NaN,NaN,NaN
18,$60.00 - $70.00 Per Hour (Employer est.),NaN,NaN,NaN
26,$49K (Employer est.),NaN,NaN,NaN


In [42]:
def process_salary(salary):
    if 'Per Hour' in salary:
        salary = salary.replace('Per Hour', '').replace('$', '').replace('K', '').replace('(Employer est.)', '').strip()
        salary = salary.split('-')
        salary = [float(s) * 2000 if '.' in s else float(s) * 1000 for s in salary]
    else:
        salary = salary.replace('$', '').replace('K', '').replace('(Employer est.)', '').strip()
        salary = salary.split('-')
        salary = [float(s) * 1000 for s in salary]
    
    if len(salary) == 1:
        salary = salary * 2

    average = np.mean(salary)
    return salary[0], salary[1], average

new_df['start'], new_df['end'], new_df['average'] = zip(*new_df['salary_estimate'].map(process_salary))

In [43]:
df.loc[new_df.index, ["start","end", "average"]] = new_df[["start","end", "average"]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   job_title         820 non-null    object        
 1   company_name      820 non-null    object        
 2   location          820 non-null    object        
 3   days_posted       820 non-null    object        
 4   salary_estimate   719 non-null    object        
 5   job_description   820 non-null    object        
 6   company_rating    820 non-null    float64       
 7   company_size      820 non-null    object        
 8   company_founded   820 non-null    int32         
 9   company_type      820 non-null    object        
 10  company_industry  820 non-null    object        
 11  company_sector    820 non-null    object        
 12  company_revenue   820 non-null    object        
 13  post_date         820 non-null    datetime64[ns]
 14  age               820 non-

In [44]:
# List of technologies you want to check for
technologies = ['Python', 'R', 'PyTorch', 'TensorFlow', 'SQL', 'Java', 'Scala', 'C\\+\\+', 'Hadoop', 'Spark', 'Tableau', 'Power BI', "mongo","Scikit-Learn", "AWS", "Azure", "GCP", "BASH", "SAP", "Git", "Docker", "Keras", "excel", "snowflake"]

# Add columns for each technology in the DataFrame
for tech in technologies:
    df[tech] = df['job_description'].str.contains(tech, case=False, regex=True).astype(int)

In [45]:
df.head()

,job_title,company_name,location,days_posted,salary_estimate,job_description,company_rating,company_size,company_founded,company_type,...,AWS,Azure,GCP,BASH,SAP,Git,Docker,Keras,excel,snowflake
0,Junior Data Analyst Apprentice,Evergreen Trading,"New York, NY",22d,$45K (Employer est.),About Evergreen\nEvergreen Trading is a media ...,3.4,Unknown,0,Company - Private,...,0,0,0,0,0,1,0,0,1,0
1,Data Analyst,Viant,"Irvine, CA",5d,$57K - $70K (Employer est.),WHAT YOU’LL DO\n\nCome help us build Viant’s i...,4.0,501 to 1000 Employees,1999,Company - Public,...,1,0,0,0,0,1,0,0,0,0
2,Sportsbook Data Analyst,DraftKings,"Boston, MA",24h,$77K - $115K (Employer est.),We’re defining what it means to build and deli...,4.0,1001 to 5000 Employees,2012,Company - Public,...,0,0,0,0,0,1,0,0,0,0
3,Junior Data Analyst,391 Financial,"Columbia, MO",30d+,$46K - $75K (Glassdoor est.),Junior Data Analyst Responsibilities:\nInterpr...,3.0,1 to 50 Employees,0,Contract,...,0,0,0,0,0,0,0,0,0,0
4,Junior Data Analyst,"Paul, Weiss, Rifkind, Wharton & Garrison LLP",United States,24h,$70K - $85K (Employer est.),Job Description\nJunior Data Analyst will be r...,4.0,1001 to 5000 Employees,1946,Private Practice / Firm,...,0,0,0,0,0,0,0,0,1,0


In [46]:
# Define keywords for each category
data_science_keywords = ['Data Scientist', 'Data Science', 'DS', 'Scientist']
data_analytics_keywords = ['Data Analyst', 'Data Analytics', 'DA']
machine_learning_keywords = ['Machine Learning', 'ML', 'AI Engineer']

# Function to categorize job titles
def categorize_job_title(title):
    title = title.lower()
    if any(keyword.lower() in title for keyword in data_science_keywords):
        return 'Data Science'
    elif any(keyword.lower() in title for keyword in data_analytics_keywords):
        return 'Data Analytics'
    elif any(keyword.lower() in title for keyword in machine_learning_keywords):
        return 'Machine Learning Engineer'
    else:
        return 'Other'

# Create a new column 'job_category'
df['job_category'] = df['job_title'].apply(categorize_job_title)

df.job_category.value_counts()


job_category
Data Analytics    717
Other              88
Data Science       15
Name: count, dtype: int64

In [47]:
df.iloc[:, :18].head()

,job_title,company_name,location,days_posted,salary_estimate,job_description,company_rating,company_size,company_founded,company_type,company_industry,company_sector,company_revenue,post_date,age,start,end,average
0,Junior Data Analyst Apprentice,Evergreen Trading,"New York, NY",22d,$45K (Employer est.),About Evergreen\nEvergreen Trading is a media ...,3.4,Unknown,0,Company - Private,Investment & Asset Management,Financial Services,Unknown / Non-Applicable,2023-12-01,2023,45000.0,45000.0,45000.0
1,Data Analyst,Viant,"Irvine, CA",5d,$57K - $70K (Employer est.),WHAT YOU’LL DO\n\nCome help us build Viant’s i...,4.0,501 to 1000 Employees,1999,Company - Public,Advertising & Public Relations,Media & Communication,Unknown / Non-Applicable,2023-12-18,24,57000.0,70000.0,63500.0
2,Sportsbook Data Analyst,DraftKings,"Boston, MA",24h,$77K - $115K (Employer est.),We’re defining what it means to build and deli...,4.0,1001 to 5000 Employees,2012,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable,2023-12-23,11,77000.0,115000.0,96000.0
3,Junior Data Analyst,391 Financial,"Columbia, MO",30d+,$46K - $75K (Glassdoor est.),Junior Data Analyst Responsibilities:\nInterpr...,3.0,1 to 50 Employees,0,Contract,--,--,Unknown / Non-Applicable,2023-11-23,2023,46000.0,75000.0,60500.0
4,Junior Data Analyst,"Paul, Weiss, Rifkind, Wharton & Garrison LLP",United States,24h,$70K - $85K (Employer est.),Job Description\nJunior Data Analyst will be r...,4.0,1001 to 5000 Employees,1946,Private Practice / Firm,Legal,Legal,$100 to $500 million (USD),2023-12-23,77,70000.0,85000.0,77500.0
